# What is a database index

It's a data structure used to quickly locate data in a database table without having to search through all rows. It uses extra space in the disk (used to store the indexes) in order to increase speed performance. A single table can have multiple indexes, each one can be associated to one or more columns of that table.

# How to create an index in Django

Index creation on Django is done when defining the model class. Although there are multiple ways to create an index, the simplest one is to us pass the `db_index=True` kwarg in the definition of a model field.

You can also set a list of `indexes` inside a model `Meta` class and define which field(s) each index will use, as in the example below. More on the different types of indexes later.

```python
from django.db import models

class MyModel(models.Model):
    foo = models.CharField(max_length=255)
    bar = models.CharField(max_length=255)
    class Meta:
        indexes = [
            models.Index(fields=['foo']),
            models.Index(fields=['bar']),
        ]
```

However, Django may automatically create some indexes, even if not explicitly asked to, to optimize for the most common cases. Let's consider the following scenario:

```python
from django.db import models

class Team(models.Model):
    name = models.CharField(max_length=255)
    tv_name = models.CharField(max_length=4, unique=True)
    city = models.CharField(max_length=255)

class Player(models.Model):
    name = models.CharField(max_length=255)
    position = models.CharField(max_length=2)
    team = models.ForeignKey(Team, on_delete=models.CASCADE)

class Stadium(models.Model):
    name = models.CharField(max_length=255)
    nickname = models.CharField(max_length=255)
    teams = models.ManyToManyField(Team)
```

In [1]:
%%sh
django-admin sqlmigrate teams 0001

BEGIN;
--
-- Create model Team
--
CREATE TABLE "teams_team" ("id" bigserial NOT NULL PRIMARY KEY, "name" varchar(255) NOT NULL, "tv_name" varchar(4) NOT NULL UNIQUE, "city" varchar(255) NOT NULL);
--
-- Create model Stadium
--
CREATE TABLE "teams_stadium" ("id" bigserial NOT NULL PRIMARY KEY, "name" varchar(255) NOT NULL, "nickname" varchar(255) NOT NULL);
CREATE TABLE "teams_stadium_teams" ("id" bigserial NOT NULL PRIMARY KEY, "stadium_id" bigint NOT NULL, "team_id" bigint NOT NULL);
--
-- Create model Player
--
CREATE TABLE "teams_player" ("id" bigserial NOT NULL PRIMARY KEY, "name" varchar(255) NOT NULL, "position" varchar(2) NOT NULL, "team_id" bigint NOT NULL);
CREATE INDEX "teams_team_tv_name_7d091fee_like" ON "teams_team" ("tv_name" varchar_pattern_ops);
ALTER TABLE "teams_stadium_teams" ADD CONSTRAINT "teams_stadium_teams_stadium_id_team_id_0174a71c_uniq" UNIQUE ("stadium_id", "team_id");
ALTER TABLE "teams_stadium_teams" ADD CONSTRAINT "teams_stadium_teams_stadium_id_1521a159_

By running `sqlmigrate`, we can inspect the SQL that Django will execute to create the table.

Notice how it calls `CREATE INDEX` multiple times, even though we never explicitly define a single index for any of our fields.

```sql
CREATE INDEX "teams_stadium_teams_stadium_id_1521a159" ON "teams_stadium_teams" ("stadium_id");
CREATE INDEX "teams_stadium_teams_team_id_57c6bc0d" ON "teams_stadium_teams" ("team_id");
CREATE INDEX "teams_player_team_id_4ee5cf70" ON "teams_player" ("team_id");
```

The indexes above are automatically created to optimize the queries we make to access our one-to-many/many-to-many fields. You c

There's another interesting index there that's used for the `tv_name` field.
```sql
CREATE INDEX "teams_team_tv_name_7d091fee_like" ON "teams_team" ("tv_name" varchar_pattern_ops);
```

This last index is created because we defined `tv_name` as `unique=True` directly in the field declaration. It will use the `varchar_pattern_ops` PostgreSQL function to try to pattern match incoming values with the values stored in the database. Maybe you don't need that extra optimization. In our case, `tv_name` is a short string (only 4 characters) and represents the name of the teams that will be displayed in the TV broadcast of a football match between them (i.e.: MUFC for Manchester United, RMAD for Real Madrid, ROMA for Roma, etc). We'll talk more about this index when discussing the tradeoffs.

In this context of short names where we can even have names that are almost the same, maybe running a full equality comparison is enough in terms of speed, and can save us some valuable database space.

As we still want `tv_name` to be unique, we can remove `unique=True` from the field and create a unique constraint in the model `Meta` class. For this presentation, we'll create a new field on our model (`tv_name_without_unique`) so we can compare performance with the original field.

```python
class Team(models.Model):
    name = models.CharField(max_length=255)

    tv_name = models.CharField(max_length=4, unique=True)
    tv_name_without_unique = models.CharField(max_length=4)

    city = models.CharField(max_length=255)

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=['tv_name_without_unique'],
                name='%(app_label)s_%(class)s_tv_name_unique',
            ),
        ]
```

In [2]:
%%sh
django-admin sqlmigrate teams 0002

BEGIN;
--
-- Add field tv_name_without_unique to team
--
ALTER TABLE "teams_team" ADD COLUMN "tv_name_without_unique" varchar(4) DEFAULT '' NOT NULL;
ALTER TABLE "teams_team" ALTER COLUMN "tv_name_without_unique" DROP DEFAULT;
--
-- Create constraint teams_team_tv_name_unique on model team
--
ALTER TABLE "teams_team" ADD CONSTRAINT "teams_team_tv_name_unique" UNIQUE ("tv_name_without_unique");
COMMIT;


Let's now populate our DB with some data, so we can compare the performance when querying these 2 fields.

In [4]:
from common import scripts

scripts.clear()

scripts.run()

Created 10000 teams...
Created 20000 teams...
Created 30000 teams...
Created 40000 teams...
Created 50000 teams...
Created 60000 teams...
Created 70000 teams...
Created 80000 teams...
Created 90000 teams...
Created 100000 teams...
Created 110000 teams...
Created 120000 teams...
Created 130000 teams...
Created 140000 teams...
Created 150000 teams...
Created 160000 teams...
Created 170000 teams...
Created 180000 teams...
Created 190000 teams...
Created 200000 teams...
Created 210000 teams...
Created 220000 teams...
Created 230000 teams...
Created 240000 teams...
Created 250000 teams...
Created 260000 teams...
Created 270000 teams...
Created 280000 teams...
Created 290000 teams...
Created 300000 teams...
Created 310000 teams...
Created 320000 teams...
Created 330000 teams...
Created 340000 teams...
Created 350000 teams...
Created 360000 teams...
Created 370000 teams...
Created 380000 teams...
Created 390000 teams...
Created 400000 teams...
Created 410000 teams...
Created 420000 teams...
C

In [5]:
from teams.models import Team

Team.objects.count()

456976

# Tradeoffs

Let's first see what are the sizes of the index and constraints created for the `tv_name`-like columns. We can use the database functions `pg_size_pretty` and `pg_relation_size` to help us.

In [10]:
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute(
        """
        select
            pg_size_pretty(pg_relation_size('teams_team_tv_name_7d091fee_like')),
            pg_size_pretty(pg_relation_size('teams_team_tv_name_key')),
            pg_size_pretty(pg_relation_size('teams_team_tv_name_unique'))
        """
    )
    row = cursor.fetchone()

row

('10040 kB', '10040 kB', '10040 kB')

For a table with 456.976 rows, Django allocated around 10MB only for each of them. You can imagine this number will grow even further if our table has more rows, which is a safe assumption when thinking of real world scenarios.

Now let's compare how much time it would take for Django to query using the `tv_name` column in both versions. We'll use the `.explain(analyze=True)` to help us inspecting what'll be the strategy used to search the database.

In [8]:
from teams.models import Team

print(Team.objects.filter(tv_name="MUFC").explain(analyze=True))

print()

print(Team.objects.filter(tv_name_without_unique="MUFC").explain(analyze=True))

Index Scan using teams_team_tv_name_7d091fee_like on teams_team  (cost=0.42..8.44 rows=1 width=72) (actual time=0.040..0.042 rows=1 loops=1)
  Index Cond: ((tv_name)::text = 'MUFC'::text)
Planning Time: 0.105 ms
Execution Time: 0.124 ms

Index Scan using teams_team_tv_name_unique on teams_team  (cost=0.42..8.44 rows=1 width=72) (actual time=0.027..0.028 rows=1 loops=1)
  Index Cond: ((tv_name_without_unique)::text = 'MUFC'::text)
Planning Time: 0.067 ms
Execution Time: 0.089 ms


We can see the difference in performance is pretty much negligible, since PostgreSQL will use the uniqueness constraint that has been added to `tv_name_without_unique` to improve performance.

Now considering the scenarios and the indexes/constraints that have been created:


- `tv_name = models.CharField(max_length=4, unique=True)`
 - `teams_team_tv_name_key` (10MB)
 - `teams_team_tv_name_7d091fee_like` (10MB)


- `tv_name_without_unique = models.CharField(max_length=4)`
 - `teams_team_tv_name_unique` (10MB)


It's safe to assume that the best approach is to create the field in the way we did with `tv_name_without_unique`, was the difference in performance is pretty much none and we avoid filling more space in disk.

# Further optimizations

Let's now try some other optimizations in our models. First of all, we'll add an index to the `name` field on our `Team` model, which will end up looking like this:

```python
class Team(models.Model):
    name = models.CharField(max_length=255)
    name_with_index = models.CharField(max_length=255, db_index=True)

    tv_name = models.CharField(max_length=4)
    tv_name_without_unique = models.CharField(max_length=4)

    city = models.CharField(max_length=255)

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=['tv_name_without_unique'],
                name='%(app_label)s_%(class)s_tv_name_unique',
            ),
        ]
```

In [11]:
%%sh
django-admin sqlmigrate teams 0003

BEGIN;
--
-- Add field name_with_index to team
--
ALTER TABLE "teams_team" ADD COLUMN "name_with_index" varchar(255) DEFAULT '' NOT NULL;
ALTER TABLE "teams_team" ALTER COLUMN "name_with_index" DROP DEFAULT;
CREATE INDEX "teams_team_name_with_index_1593f8bc" ON "teams_team" ("name_with_index");
CREATE INDEX "teams_team_name_with_index_1593f8bc_like" ON "teams_team" ("name_with_index" varchar_pattern_ops);
COMMIT;


By adding the `db_index=True` to the `name`-like column, that same `varchar_pattern_ops` index from before was created. This time, it may come useful, as pattern matching on clubs names can come handy, especially as they can be pretty large.

Let's measure the impact of this change. The first query will be performed over the column with an index and the second will be done on the original column.

In [12]:
from teams.models import Team

print(Team.objects.filter(name_with_index="abcdeFGHijkl").explain(analyze=True))

print()

print(Team.objects.filter(name="abcdeFGHijkl").explain(analyze=True))

Index Scan using teams_team_name_with_index_1593f8bc_like on teams_team  (cost=0.42..8.44 rows=1 width=72) (actual time=0.022..0.022 rows=0 loops=1)
  Index Cond: ((name_with_index)::text = 'abcdeFGHijkl'::text)
Planning Time: 0.075 ms
Execution Time: 0.057 ms

Gather  (cost=1000.00..9233.31 rows=1 width=72) (actual time=15.206..17.381 rows=0 loops=1)
  Workers Planned: 2
  Workers Launched: 2
  ->  Parallel Seq Scan on teams_team  (cost=0.00..8233.21 rows=1 width=72) (actual time=12.838..12.838 rows=0 loops=3)
        Filter: ((name)::text = 'abcdeFGHijkl'::text)
        Rows Removed by Filter: 152325
Planning Time: 0.086 ms
Execution Time: 17.425 ms


The difference is pretty visible here, with the query over the indexed field being over 300x more performatic. For this scenario, we must start analyzing the tradeoffs, as we can see in the migration that 2 new indexes have been created (meaning space in disk being allocated).

In [14]:
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute(
        """
        select
            pg_size_pretty(pg_relation_size('teams_team_name_with_index_1593f8bc')),
            pg_size_pretty(pg_relation_size('teams_team_name_with_index_1593f8bc_like'))
        """
    )
    row = cursor.fetchone()

row

('18 MB', '18 MB')

## Partial indexes

Now let's imagine we work for Globe, a company that just bought the broadcasting rights for the Zora's Domain Football Cup. We then decide to query all the teams that are from that city, so we can build a special page on our website just for that competition.

In [15]:
from teams.models import Team

Team.objects.filter(city="Zora's Domain").explain(analyze=True)

"Seq Scan on teams_team  (cost=0.00..11565.50 rows=45700 width=72) (actual time=0.040..38.127 rows=45547 loops=1)\n  Filter: ((city)::text = 'Zora''s Domain'::text)\n  Rows Removed by Filter: 411429\nPlanning Time: 0.137 ms\nExecution Time: 39.246 ms"

Around 40ms is certainly "fine" in terms of performance, but can we do better? The answer is YES.

There's a special type of index named Partial Index. It allows us to index a subset of our database that we know will be used a lot in our queries. You can, i.e.:, index a subset of teams that are from the "Zora's Domain" city.

```python
class Team(models.Model):
    name = models.CharField(max_length=255)
    name_with_index = models.CharField(max_length=255, db_index=True)

    tv_name = models.CharField(max_length=4, unique=True)
    tv_name_without_unique = models.CharField(max_length=4)

    city = models.CharField(max_length=255)
    city_with_partial_index = models.CharField(max_length=255)

    class Meta:
        constraints = [
            models.UniqueConstraint(
                fields=['tv_name_without_unique'],
                name='%(app_label)s_%(class)s_tv_name_unique',
            ),
        ]
        indexes = [
            models.Index(
                fields=['city_with_partial_index'],
                condition=models.Q(city_with_partial_index="Zora's Domain"),
                name='%(app_label)s_%(class)s_city_zora',
            ),
        ]
```

We do so by adding a new index to the newly created `city`-like field, that will internally "create a subset" of rows that fit a given `condition` (in our case, `city == "Zora's Domain"`).

Inspecting the migration, we see that PostgreSQL created an index for us that does exactly that.

In [16]:
%%sh
django-admin sqlmigrate teams 0004

BEGIN;
--
-- Add field city_with_partial_index to team
--
ALTER TABLE "teams_team" ADD COLUMN "city_with_partial_index" varchar(255) DEFAULT '' NOT NULL;
ALTER TABLE "teams_team" ALTER COLUMN "city_with_partial_index" DROP DEFAULT;
--
-- Create index teams_team_city_zora on field(s) city_with_partial_index of model team
--
CREATE INDEX "teams_team_city_zora" ON "teams_team" ("city_with_partial_index") WHERE "city_with_partial_index" = 'Zora''s Domain';
COMMIT;


Now let's compare the results:

In [17]:
from teams.models import Team

Team.objects.filter(city_with_partial_index="Zora's Domain").explain(analyze=True)

"Bitmap Heap Scan on teams_team  (cost=392.20..6816.45 rows=45700 width=72) (actual time=3.091..16.218 rows=45547 loops=1)\n  Recheck Cond: ((city_with_partial_index)::text = 'Zora''s Domain'::text)\n  Heap Blocks: exact=5848\n  ->  Bitmap Index Scan on teams_team_city_zora  (cost=0.00..380.78 rows=45700 width=0) (actual time=2.425..2.426 rows=45547 loops=1)\nPlanning Time: 0.119 ms\nExecution Time: 17.378 ms"

Just by creating this index, we're able to reduce the query time in more than half of the original one. Great news!

And if we compare this result with another query on the same table and column, but looking for a different city (i.e.: `Tarrey Town`), we'll see that the query has similar performance (and planning) to the not-indexed column from the legacy table.

In [20]:
from teams.models import Team

print(Team.objects.filter(city="Tarrey Town").explain(analyze=True))

print()

print(Team.objects.filter(city_with_partial_index="Tarrey Town").explain(analyze=True))

Seq Scan on teams_team  (cost=0.00..11565.50 rows=47635 width=72) (actual time=0.035..36.957 rows=45397 loops=1)
  Filter: ((city)::text = 'Tarrey Town'::text)
  Rows Removed by Filter: 411579
Planning Time: 0.051 ms
Execution Time: 38.090 ms

Seq Scan on teams_team  (cost=0.00..11565.50 rows=47635 width=72) (actual time=0.016..39.489 rows=45397 loops=1)
  Filter: ((city_with_partial_index)::text = 'Tarrey Town'::text)
  Rows Removed by Filter: 411579
Planning Time: 0.051 ms
Execution Time: 40.523 ms


Now let's check the space in disk that PostgreSQL took to create that index. Around 300kB, which is also great news. That's an optimization without much tradeoffs, that can be safely implemented if we have a scenario that we know we're going to repeatedly query a lot.

In [22]:
from django.db import connection

with connection.cursor() as cursor:
    cursor.execute(
        """
        select
            pg_size_pretty(pg_relation_size('teams_team_city_zora'))
        """
    )
    row = cursor.fetchone()

row

('304 kB',)